# Generate compound conditioning on profiles from the training set
Unlike other molecule generation scripts, here we fix the number of generation tries and not the number of valid/unique generated molecules since we want to get stats on the validity and uniqueness of the generation process

In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
import tensorflow.keras.backend as K
import logging 
import inference as infr
import utils

from sklearn.preprocessing import QuantileTransformer
from rdkit import Chem
from rdkit.Chem.Descriptors import qed
import sys
sys.path.append(os.path.join(Chem.RDConfig.RDContribDir, 'SA_Score'))
import sascorer


from importlib import reload
reload(infr)

<module 'inference' from '/gpfs01/home/ghpxd/CellPainting_GAN/git_repo/cellpainting_for_compound_generation/Conditioned_GAN/inference.py'>

### Arguments

In [2]:
args = {
    "Nmols_total":30000,
    "Nmols_per_condition": 1,
    "use_median_profile":False,
    "shuffle_features":False,
    "conditon_subgroup": ['Cpds','DMSO',"CpdsAndDMSO"][0],
    'gpu': '7',
    'filename_train_profiles':'Data/Compound_dataset/CellPainting_30kcpds_normalized_profiles__final.csv.gz',
    'filename_train_MolEmbeddings':'Data/Compound_dataset/CellPainting_30kcpds__molecular_embeddings__unique_SMILES.csv',
    'filename_quantile_transformer':'Trained_models/quantile_transformer.pkl',
    'model_weight_paths':{
        'autoencoder': 'Trained_models/Selfies_Autoencoder/selfies_EncoderDecoder_epoch0010.h5',
        'wgan':{
            'C': 'Trained_models/WGAN/wgan_C_500epochs.h5',
            'D': 'Trained_models/WGAN/wgan_D_500epochs.h5',
            'G1':'Trained_models/WGAN/wgan_G1_500epochs.h5',
            'condition_encoder':'Trained_models/WGAN/wgan_condition_encoder_500epochs.h5',
            'classifier':'Trained_models/WGAN/wgan_classifier.h5',
        }
    },
    "output_dir":"Results/train_rand__GeneratedMols_COND_SHUFF/generated_mols/"
}

# Output file and directories automatic naming
if args["use_median_profile"]: args["output_dir"]=args["output_dir"].replace("COND","CondMedianProfile")
else: args["output_dir"]=args["output_dir"].replace("COND","CondSingleProfiles")
args["output_dir"] = args["output_dir"].replace("_SHUFF","_ShuffledFeatures"*args['shuffle_features'])

if not os.path.isdir(args["output_dir"]):
    os.makedirs(args["output_dir"])
args["output_dir"]

'Results/train_rand__GeneratedMols_CondSingleProfiles/generated_mols/'

### logging and GPU set up

In [3]:
logging.basicConfig(level=logging.INFO, format ='%(levelname)s - %(message)s')
tf.logging.set_verbosity(tf.logging.ERROR)

os.environ['CUDA_VISIBLE_DEVICES'] = str(args['gpu'])
gpu_options = tf.GPUOptions(visible_device_list='0')
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
K.set_session(sess)
tf.config.set_soft_device_placement(True)
tf.debugging.set_log_device_placement(True)

## 1. Load inference model and quantile transformer

In [4]:
quantile_transformer =  pickle.load( open( args['filename_quantile_transformer'], 'rb' ) )
model = infr.InferenceModel( args['model_weight_paths'] ) 

## 2. Read profiles and apply transformer
Takes long ...

In [5]:
data_train = pd.read_csv(args['filename_train_profiles'], index_col=0 )
data_train = data_train.rename(columns={"SMILES_standard":"SMILES_standard_conditioned"})

feature_cols , _ = utils.get_feature_cols(data_train)
data_train[feature_cols] = quantile_transformer.transform(data_train[feature_cols].values) 
print('Number of Morphological profiles: %i'%data_train.shape[0])

Number of Morphological profiles: 153351


In [6]:
data_train.head()

,Metadata_broad_sample,Metadata_Plate,Metadata_Well,BROAD_ID,CPD_NAME,SMILES_standard_conditioned,structure,Cells_AreaShape_Area,Cells_AreaShape_Center_X,Cells_AreaShape_Center_Y,...,Nuclei_Texture_Variance_DNA_5_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_ER_3_0,Nuclei_Texture_Variance_ER_5_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_Mito_3_0,Nuclei_Texture_Variance_Mito_5_0,Nuclei_Texture_Variance_RNA_10_0,Nuclei_Texture_Variance_RNA_3_0,Nuclei_Texture_Variance_RNA_5_0
0,DMSO,24512.0,a01,NaN,NaN,NaN,NaN,0.919258,1.835465,-0.407943,...,-0.455231,0.613343,0.475295,0.300097,-1.024866,-1.048468,-1.151256,-0.492058,-0.565263,-0.318163
1,DMSO,24512.0,a02,NaN,NaN,NaN,NaN,-0.312693,1.525036,-0.793711,...,-0.661935,-0.022998,0.238424,0.267652,-1.284063,-0.997971,-1.059767,-0.405197,-0.375833,-0.151537
2,BRD-K58665933-001-06-5,24512.0,a03,BRD-K58665933-001-06-5,BRD-K58665933,Nc1nc(Nc2cccc(F)c2)nc(NCc2ccco2)c1[N+](=O)[O-],Nc1nc(Nc2cccc(F)c2)nc(NCc2ccco2)c1[N+](=O)[O-],0.056372,0.433012,-0.690138,...,-0.037428,0.882511,-0.335781,0.085200,-0.801905,-1.773358,-1.421458,-0.823743,-1.225657,-1.071240
3,BRD-K14750527-001-07-8,24512.0,a04,BRD-K14750527-001-07-8,BRD-K14750527,CCNc1nc(NC(C)C)nc(NC(C)(C)C)n1,CCNc1nc(NC(C)C)nc(NC(C)(C)C)n1,-0.429885,0.042910,0.478025,...,-0.098636,-1.995082,-0.850405,-0.985754,-1.275041,-1.156870,-1.024981,-0.414297,-0.861382,-0.977865
4,BRD-K58826199-001-05-4,24512.0,a05,BRD-K58826199-001-05-4,BRD-K58826199,Cc1c(Br)c(=O)oc2c(C)c(O)ccc12,Cc1c(Br)c(=O)oc2c(C)c(O)ccc12,0.381937,0.771178,0.735148,...,-0.021834,-1.223184,-1.016891,-1.160295,-0.634518,-0.932297,-0.862763,-0.480909,-0.675575,-0.479456


## 3. Select data subset:

In [7]:
# Compounds and/or DMSO
if args['conditon_subgroup'] == 'Cpds':
    data = data_train[ data_train['Metadata_broad_sample'] !='DMSO' ].reset_index(drop=True)
elif args['conditon_subgroup'] == 'DMSO':
    data = data_train[ data_train['Metadata_broad_sample'] =='DMSO'].reset_index(drop=True)
else:
    data = data_train
print('Number of selected samples: %i'%len(data))
print('Number of unique cpds : %i'%len(data.SMILES_standard_conditioned.unique()))

# If compounds are included, only use profiles from molecules with a valid molecular embedding
# Since some molecules do not have a valid selifes conversion
if args['conditon_subgroup'] != 'DMSO':
    embd_df = pd.read_csv(args['filename_train_MolEmbeddings'],index_col=0) 
    data = data.loc[ data.SMILES_standard_conditioned.isin(embd_df.SMILES_standard)].reset_index(drop=True)
    print("\nAfter removing cpds with invalid molecular embeddings")
print('Number of selected samples: %i'%len(data))
print('Number of unique cpds : %i'%len(data.SMILES_standard_conditioned.unique()))
    

Number of selected samples: 126779
Number of unique cpds : 15413

After removing cpds with invalid molecular embeddings
Number of selected samples: 117424
Number of unique cpds : 13448


## 4. Calculate median profile or shuffle features if asked 

In [8]:
if args["use_median_profile"]:
    logging.info("Calculating mean profiles")
    data = data[ ["Metadata_broad_sample"] + feature_cols ].groupby(by="Metadata_broad_sample").median().reset_index(drop=False) # all other cols dont apply anymore 
else: 
    data = data[ ["Metadata_Plate","Metadata_Well","Metadata_broad_sample","SMILES_standard_conditioned"] + feature_cols  ]

feature_cols , info_cols = utils.get_feature_cols(data)
if args['shuffle_features']:
    logging.info("Shuffling features")
    data[feature_cols] = data[feature_cols].sample(frac=1).values # Do not remove values. It will align the indexes and not shuffle 


## 5. Select random subset of profiles

In [9]:
# Randomly sample 2 profiles per SMILES standard
if args['Nmols_total']==None:
    print("Selecting Two cpd per smiles")
    
    def sample_2(x):
        if len(x)<2: return x
        else: return x.sample(2,random_state=10)
        
    rand_subset = data.groupby(by='SMILES_standard_conditioned').apply(lambda x: sample_2(x)).reset_index(drop=True)
    Nmols_str = '2ProfPerUniqueSmile_'+str(args['Nmols_per_condition'])+'PerCond'

# Randomly sample a given number of profiles without caring about its SMILES    
else:
    Nconditions = int(np.ceil( args['Nmols_total']/args['Nmols_per_condition'] ))
    rand_subset = data.sample(Nconditions, random_state=10).reset_index(drop=True)
    Nmols_str = str(args["Nmols_total"])
print("Using %i profiles for molecule generation"%len(rand_subset))

Using 30000 profiles for molecule generation


## 6. Generate molecules

In [11]:
# Define output file and check that it doesnt aleady exits
output_file = os.path.join(args["output_dir"], args['conditon_subgroup']+'__'+Nmols_str+".csv")

if not os.path.isfile(output_file):

    # Generate molecules
    generated = infr.generate_compounds_multiple_conditions( model, rand_subset, feature_cols, info_cols, seed=10, nsamples = args['Nmols_per_condition'])

    # Standardize SMILES and check validity and other properties
    generated["SMILES_standard"]= utils.clean_smiles_parallel( generated.SMILES )
    generated['valid'] = generated.SMILES_standard.isnull()==False
    idx = generated['valid'] == True
    physchem_filterer = utils.PhysChemFilters()
    generated.loc[idx, "pass_physchem_filter"] =  generated.loc[idx, "SMILES_standard"].apply(lambda x: physchem_filterer.apply_filters(x))
    rdkit_mols = [ Chem.MolFromSmiles(x) for x in generated.loc[idx, 'SMILES_standard'] ]
    generated.loc[idx, "SA_score"]  = [ sascorer.calculateScore(x) for x in rdkit_mols]
    generated.loc[idx, "QED_score"] = [ qed(x) for x in rdkit_mols]

    # Save results
    logging.info("Saving to %s"%output_file)
    generated.to_csv(output_file)

else:
    print("File %s already exists. Please remove it to re-wrtite it"%output_file)

File Results/train_rand__GeneratedMols_CondSingleProfiles/generated_mols/Cpds__30000.csv already exists. Please remove it to re-wrtite it
